# LangSmith로 프롬프트 개선

### 1. 뉴스 요약하기

* 라이브러리 호출 및 프로젝트 이름 초기화

In [1]:
import os
import bs4
import uuid
from dotenv import load_dotenv
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.llms import Ollama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langsmith import Client
from langchain_community.llms import Ollama
from langchain_ollama import ChatOllama

load_dotenv()
# 1-1. LangSmith project, dataset, annotation queue 이름으로 사용할 문자열
project_name = "prompt_enhance_example"

USER_AGENT environment variable not set, consider setting it to identify your requests.


* 크롤링 해올 뉴스 링크 초기화 및 LangSmith 플랫폼과 로컬 LLM이 있는 PC와의 상호작용하기 위한 Langsmith Client 생성 

In [2]:
# 1-2. 테스트 대상이 되는 뉴스 URLs
news_urls = [
    """https://www.bbc.com/korean/articles/c166p510n79o""",
]
# 1-3. langsmith client 생성
client = Client()

c:\Users\IT\anaconda3\envs\LG_LLM\lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


* 뉴스 요약용 프롬프트 생성
    * LangSmith로 프롬프트를 개선하려면 “예제(뉴스 요약)”과 “이에 대한 피드백을 통한 프롬프트 개선” 총 2번 LLM을 사용함
    * “뉴스 요약용” 프롬프트와 “프롬프트 개선용” 프롬프트 두개가 필요
    * 뉴스 요약 프롬프트에서 지시문이 들어갈 부분을 instruction 변수로 밖으로 빼서 이후에 이 변수 부분만 업데이트하고 나머지 전체 형식은 유지


In [3]:
# 1-4. 뉴스 요약 PROMPT Template 생성
instruction = """당신의 임무는 주어진 뉴스(news_text)에서 기사제목(title), 작성자(author), 
작성일자(date), 요약문(summary) - 4가지 항목을 추출하는 것입니다.
결과는 한국어로 작성해야합니다. 뉴스와 관련성이 높은 내용만 포함하고 추측된 내용을 생성하지 마세요.
"""
prompt = PromptTemplate.from_template(
    """당신은 뉴스 기사를 요약, 정리하는 AI 어시스턴트입니다. 
     {instruction}

<news_text>
{news}
<news_text>

요약된 결과는 아래 형식에 맞춰야합니다.
<news>
  <title></title>
  <author></author>
  <date></date>
  <summary></summary>
</news>
"""
)

* 프롬프트 개선용 프롬프트 생성

In [4]:
# 1-5. PROMPT 최적화용 Template 생성
optimizer_prompt = PromptTemplate.from_template(
    """당신은 AI 프롬프트 전문가입니다. 
아래 뉴스 요약 프롬프트(<prompt>)가 있습니다.
<prompt>
{instruction}
</prompt>

그리고 결과의 만족도를 평가한 피드백(<feedbacks>)이 주어집니다.
<feedbacks>
{human_feedback}
</feedbacks>

당신은 이 피드백을 참고해서 기존의 프롬프트(<prompt>)를 개선해야 합니다.
개선된 프롬프트는 <newprompt></newprompt> 태그 사이에 넣어주세요.
"""
)


* WebBaseLoader: LangChain의 웹 문서 로더로, URL 목록에 있는 웹페이지를 불러옴
* bs_kwargs : BeautifulSoup라는 Python에서 HTML, XML 문서로부터 쉽게 데이터를 추출할 수 있게 돕는 라이브러리를 쓸 때 파라미터를 설정 div 태그 중 class가 bbc-1cvxiy9, bbc-fa0wmp인 것만 필터링함은 의미

In [ ]:
# 1-5. 뉴스 스크래핑 (Document Loading)
loader = WebBaseLoader(
    web_paths=(news_urls),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["bbc-1cvxiy9", "bbc-fa0wmp"]},
        )
    ),
)
news_array = loader.load()

# 1-6. 뉴스 원문을 저장할 dataset 생성 또는 가져오기
try:
    ds = client.create_dataset(dataset_name=project_name)
    print(f"----- New dataset created: {ds}")
except Exception as e:
    # 이미 존재하는 경우, 기존 dataset을 가져옵니다
    existing_datasets = list(client.list_datasets(dataset_name=project_name))
    if existing_datasets:
        ds = existing_datasets[0]
        print(f"----- Using existing dataset: {ds}")
    else:
        print(f"----- Failed to create or retrieve dataset: {e}")

LangSmithAuthError: Authentication failed for /datasets. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/datasets?limit=100&name=prompt_enhance_example&offset=0', '{"detail":"Invalid token"}')

: 

* 사용할 LLM 들을 초기화하고 뉴스 요약에는 llama 3.1 프롬프트 개선에는 exaone 3.5를 사용

In [35]:
# 1-7. 사용할 LLM 초기화
summary_llm = Ollama(model="llama3.1", temperature=0)
optimizer_llm = ChatOllama(model="exaone3.5:7.8b")

In [36]:
# 2-1. 중간중간 생성되는 데이터를 담는 변수 선언
news = news_array[0]
example_uuid = str(uuid.uuid4())

* LangSmith에서 테스트 대상이 되는 데이터(뉴스 데이터)를 담을 수 있는 구조가 dataset
* dataset에 example이라는 이름으로 테스트 데이터를 등록하는 작업
* ids는 편의상 임의의 ID(uuid)를 지정해두고 이 값을 공유해서 사용

In [37]:
# 2-2. 앞서 생성한 데이터셋에 example 추가
try:
    client.create_examples(
        ids=[example_uuid],
        inputs=[{"news": news.page_content[:1000], "instruction" : instruction}],
        dataset_id=ds.id,
    )
    print(f"----- Added new examples successfully ({example_uuid})")
except Exception as e:
    print(f"----- Failed to add example: {e}")

----- Added new examples successfully (46bb51b4-e978-4279-aecf-6e5e162cd8ad)


In [38]:
# 2-3. 프롬프트를 실행할 체인생성
summary_chain = prompt | summary_llm | StrOutputParser()

* 데이터셋에 있는 뉴스를 뉴스 요약용 체인을 이용해 요약

In [39]:
# 2-4. dataset 에 추가한 뉴스 기사(example)에 뉴스 요약 chain 적용
print("----- Run chain on new example")
res = client.run_on_dataset(
    dataset_name=project_name,
    llm_or_chain_factory=summary_chain,
)
# 2-5. run_id, 요약 결과 추출
run_ids = [result["run_id"] for result in res["results"].values()]
summary = [result["output"] for result in res["results"].values()]

----- Run chain on new example
View the evaluation results for project 'memorable-hope-29' at:
https://smith.langchain.com/o/b0bf207d-6152-41bc-8620-1d01b85129e7/datasets/78467282-9224-4cbb-a5d5-ec569626e546/compare?selectedSessions=89de1e7f-3b8e-4fc4-bdbe-2e594d70c7f4

View all tests for Dataset prompt_enhance_example at:
https://smith.langchain.com/o/b0bf207d-6152-41bc-8620-1d01b85129e7/datasets/78467282-9224-4cbb-a5d5-ec569626e546
[------------------------------------------------->] 2/2


* LangSmith에 Annotation queue를 생성
* Annotation queue 는 LangSmith 플랫폼의 기능 중 하나로, 사람이 직접 데이터에 주석 또는 피드백을 남기는 방식으로 평가나 라벨링할 수 있게 도와주는 인터페이스


In [22]:
# 2-6. 기존 annotation queue 검색, 없으면 새로 생성
existing_queues = list(client.list_annotation_queues(name=project_name))
if existing_queues:
    # 이미 존재하는 queue 반환
    q = existing_queues[0]
    print(f"\n----- Using existing annotation queue: {q.name}")
else:
    # 새로운 queue 생성
    q = client.create_annotation_queue(name=project_name)
    print(f"\n----- Created new annotation queue: {q.name}")
    
# 2-7. feedback 받기 위해 run_on_dataset 결과를 annotation queue에 추가
if example_uuid and res:
    try:
        # run 결과를 annotation queue에 추가
        client.add_runs_to_annotation_queue(queue_id=q.id, run_ids=run_ids)
        print(f"----- Added runs to the annotation queue")
    except Exception as e:
        print(f"----- Failed to add runs to the annotation queue: {e}")
else:
    print("----- No results to add to the annotation queue")


----- Created new annotation queue: prompt_enhance_example
----- Added runs to the annotation queue


### 2. LangSmith에서 피드백 하기 (PPT 강의자료 참조) 

* https://smith.langchain.com/ 이동
* LangSmith에 Annotation queue 에서 뉴스 요약 결과에 대해서 평가 및 피드백 기능을 활용함

### 3. 프롬프트 개선

* LangSmith에서 평가에 사용한 변수명에 해당하는 값을 가져옴
* 가져온 피드백을 XML 형식으로 변환 
* 출력된 내용은 LangSmith에서 읽어온 llm의 실행 id (run_id)와 피드백 내용을 나타냄

In [23]:
def combine_feedback_by_run_id(feedback_list):
    new_feedbacks = {}
    for item in feedback_list:
        run_id = str(item.run_id)
        if run_id in new_feedbacks:
            selected_feedback = new_feedbacks[run_id]
        else:
            selected_feedback = {"correctness": 0, "score": 0, "comment": ""}
            new_feedbacks[run_id] = selected_feedback

        key = item.key
        if key == "correctness":
            selected_feedback["correctness"] = item.score
        elif key == "score":
            selected_feedback["score"] = item.score
        elif key == "note":
            selected_feedback["comment"] = item.comment

    print(f"new_feedbacks = {new_feedbacks}")

    # XML 형식으로 변환
    result = []
    for _key, value in new_feedbacks.items():
        feedback_xml = f"""<feedback>
            <correctness>{value['correctness']}</correctness>
            <score>{value['score']}/5.0</score>
            <comment>{value['comment']}</comment>
            </feedback>"""
        result.append(feedback_xml)
        print(f"feedback_xml = {feedback_xml}\n\n")

    return result
    
#############################################################################
# 3. 프롬프트 최적화(optimization) 단계
# annotation queue 에서 feedback 추출 -> 피드백을 바탕으로 프롬프트 최적화
#############################################################################
# 3-1. 피드백을 가져와서 텍스트로 변환
print("Getting feedbacks -------------------------------------------------")
res = client.list_feedback(run_ids=run_ids)
print("Parsing feedbacks -------------------------------------------------")
feedback_list = combine_feedback_by_run_id(res)

Getting feedbacks -------------------------------------------------
Parsing feedbacks -------------------------------------------------
new_feedbacks = {'07d4272b-edd5-4bb2-a263-6ed819795eb1': {'correctness': 0.0, 'score': 1.0, 'comment': '서론, 본론, 결론으로 나누어서 요약해줘'}}
feedback_xml = <feedback>
            <correctness>0.0</correctness>
            <score>1.0/5.0</score>
            <comment>서론, 본론, 결론으로 나누어서 요약해줘</comment>
            </feedback>




* 개선된 Prompt의 Instruction 부분
* prompt optimizer용 prompt, llm 으로 구성된 chain에 의해 “개선 전 instruction”과 “langsmith에서 한 피드백“을 입력으로 받아 llm이 개선된 instruction을 생성하였음
* “요약문” 부분에서 피드백인＂서론, 본론, 결론 구조로 요약 해줘＂가 반영된 것을 볼 수 있음

In [24]:
# 3-2. 현재 프롬프트, 기사 원문, 요약 결과, 피드백 데이터를 사용해서
#   프롬프트 최적화 chain 실행
optimizer = optimizer_prompt | optimizer_llm | StrOutputParser()
print("Optimize prompt -------------------------------------------------")
optimized = optimizer.invoke(
    {
        "instruction": instruction,
        "human_feedback": "\n\n".join(feedback_list),
    }
)
print(f"Optimized = \n{optimized}\n")

Optimize prompt -------------------------------------------------
Optimized = 
<newprompt>
당신의 임무는 주어진 뉴스 텍스트(news_text)에서 정확하게 다음 항목들을 한국어로 추출하는 것입니다:
1. **기사 제목(Title)**: 가장 핵심적인 내용을 담은 제목을 정확히 작성해주세요.
2. **작성자(Author)**: 기사를 작성한 사람의 이름을 기입하세요.
3. **작성일자(Date)**: 기사가 발행된 날짜를 정확히 기재해주세요.
4. **요약문(Summary)**: 뉴스의 핵심 내용을 서론, 본론, 결론의 구조로 나누어 간결하게 작성해주세요. 각 부분은 다음과 같이 구분해주세요:
   - **서론**: 주요 이슈나 사건 소개
   - **본론**: 주요 세부 사항과 관련 정보
   - **결론**: 주요 시사점이나 전망

추측된 내용은 생성하지 마시고, 뉴스와 직접적으로 관련된 정보만을 포함시켜 주세요.
</newprompt>



#### 프롬프트 개선 전 요약문

In [43]:
print(summary[0])

기사 요약 결과입니다.

<news>
  <title>구글의 연이은 검색 알고리즘 업데이트...예전과 달라질 인터넷 세상</title>
  <author>토마스 저메인, BBC News</author>
  <date>2024년 5월 28일</date>
  <summary>구글은 지난 2년 동안 검색 기능을 여러 번 업데이트했다. 이 업데이트로 인해 인터넷 상의 콘텐츠가 크게 달라졌고, 구글이 웹 세계를 구할 수 있는지 파괴할 수 있는지에 대한 논쟁이 일어나고 있다.</summary>
</news>


#### 프롬프트 개선 후 요약문

In [42]:
new_summary = summary_chain.invoke({"news":news.page_content[:1000], "instruction":optimized})
print(new_summary)

<news>
  <title>구글의 연이은 검색 알고리즘 업데이트...예전과 달라질 인터넷 세상</title>
  <author>토마스 저메인, BBC News</author>
  <date>2024년 5월 28일</date>
  <summary>
    <p>서론: 구글은 최근 몇 차례 검색 기능을 업데이트했다. 이 업데이트로 인해 인터넷 상의 강력한 도구가 전례 없는 인공지능(AI) 기능을 갖추게 되었다.</p>
    <p>본론: 하우스프레시.com이라는 웹사이트는 공기청정기 리뷰를 제공하는 사이트이다. 지젤 나바로와 남편이 시작한 이 사이트는 과학적 실험을 통해 소비자들이 자신에게 맞는 공기청정기를 찾을 수 있도록 돕는다. 그러나 구글의 검색 알고리즘 업데이트로 인해 하우스프레시.com은 사업을 초토화당했다.</p>
    <p>결론: 이 사건은 구글이 추구하는 독립적인 콘텐츠 창작자들의 역할과 인터넷 상의 변화에 대한 시사점을 제공한다. 또한, 구글의 검색 알고리즘 업데이트가 소비자와 기업 모두에게 미치는 영향을 고민해야 할 필요성이 있다.</p>
  </summary>
</news>


# 실습 문제: 직접 LangSmith를 활용해 뉴스 요약 프롬프트를 개선해보자

* 프롬프트 개선 코드의 뉴스 링크를 https://www.bbc.com/korean/articles/cjqe104j8l0o 로 변경하여 요약하고, langsmith에서 직접 피드백하여 프롬프트를 개선시켜 보자. 
* (피드백: “핵심 문장을 번호를 붙여서 나열해줘”)

### 1. 뉴스 요약하기

In [25]:
# 1-2. 테스트 대상이 되는 뉴스 URLs (변경된 부분)
news_urls = [
    """https://www.bbc.com/korean/articles/cjqe104j8l0o""",  # 예시링크, 원하는 링크로 변경 가능
]

In [26]:
import os
import bs4
import uuid
from dotenv import load_dotenv
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.llms import Ollama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langsmith import Client
from langchain_community.llms import Ollama
from langchain_ollama import ChatOllama

load_dotenv()
# 1-1. LangSmith project, dataset, annotation queue 이름으로 사용할 문자열
project_name = "prompt_enhance_example2"

# 1-3. langsmith client 생성
client = Client()

# 1-4. 뉴스 요약 PROMPT Template 생성
instruction = """당신의 임무는 주어진 뉴스(news_text)에서 기사제목(title), 작성자(author), 
작성일자(date), 요약문(summary) - 4가지 항목을 추출하는 것입니다.
결과는 한국어로 작성해야합니다. 뉴스와 관련성이 높은 내용만 포함하고 추측된 내용을 생성하지 마세요.
"""
prompt = PromptTemplate.from_template(
    """당신은 뉴스 기사를 요약, 정리하는 AI 어시스턴트입니다. 
     {instruction}

<news_text>
{news}
<news_text>

요약된 결과는 아래 형식에 맞춰야합니다.
<news>
  <title></title>
  <author></author>
  <date></date>
  <summary></summary>
</news>
"""
)

# 1-5. PROMPT 최적화용 Template 생성
optimizer_prompt = PromptTemplate.from_template(
    """당신은 AI 프롬프트 전문가입니다. 
아래 뉴스 요약 프롬프트(<prompt>)가 있습니다.
<prompt>
{instruction}
</prompt>

그리고 결과의 만족도를 평가한 피드백(<feedbacks>)이 주어집니다.
<feedbacks>
{human_feedback}
</feedbacks>

당신은 이 피드백을 참고해서 기존의 프롬프트(<prompt>)를 개선해야 합니다.
개선된 프롬프트는 <newprompt></newprompt> 태그 사이에 넣어주세요.
"""
)

# 1-5. 뉴스 스크래핑 (Document Loading)
loader = WebBaseLoader(
    web_paths=(news_urls),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["bbc-1cvxiy9", "bbc-fa0wmp"]},
        )
    ),
)
news_array = loader.load()

# 1-6. 뉴스 원문을 저장할 dataset 생성 또는 가져오기
try:
    ds = client.create_dataset(dataset_name=project_name)
    print(f"----- New dataset created: {ds}")
except Exception as e:
    # 이미 존재하는 경우, 기존 dataset을 가져옵니다
    existing_datasets = list(client.list_datasets(dataset_name=project_name))
    if existing_datasets:
        ds = existing_datasets[0]
        print(f"----- Using existing dataset: {ds}")
    else:
        print(f"----- Failed to create or retrieve dataset: {e}")

# 1-7. 사용할 LLM 초기화
summary_llm = Ollama(model="llama3.1", temperature=0)
optimizer_llm = ChatOllama(model="exaone3.5:7.8b")

# 2-1. 중간중간 생성되는 데이터를 담는 변수 선언
news = news_array[0]
example_uuid = str(uuid.uuid4())

# 2-2. 앞서 생성한 데이터셋에 example 추가
try:
    client.create_examples(
        ids=[example_uuid],
        inputs=[{"news": news.page_content[:1000], "instruction" : instruction}],
        dataset_id=ds.id,
    )
    print(f"----- Added new examples successfully ({example_uuid})")
except Exception as e:
    print(f"----- Failed to add example: {e}")

# 2-3. 프롬프트를 실행할 체인생성
summary_chain = prompt | summary_llm | StrOutputParser()

# 2-4. dataset 에 추가한 뉴스 기사(example)에 뉴스 요약 chain 적용
print("----- Run chain on new example")
res = client.run_on_dataset(
    dataset_name=project_name,
    llm_or_chain_factory=summary_chain,
)
# 2-5. run_id, 요약 결과 추출
run_ids = [result["run_id"] for result in res["results"].values()]
summary = [result["output"] for result in res["results"].values()]

# 2-6. 기존 annotation queue 검색, 없으면 새로 생성
existing_queues = list(client.list_annotation_queues(name=project_name))
if existing_queues:
    # 이미 존재하는 queue 반환
    q = existing_queues[0]
    print(f"\n----- Using existing annotation queue: {q.name}")
else:
    # 새로운 queue 생성
    q = client.create_annotation_queue(name=project_name)
    print(f"\n----- Created new annotation queue: {q.name}")
    
# 2-7. feedback 받기 위해 run_on_dataset 결과를 annotation queue에 추가
if example_uuid and res:
    try:
        # run 결과를 annotation queue에 추가
        client.add_runs_to_annotation_queue(queue_id=q.id, run_ids=run_ids)
        print(f"----- Added runs to the annotation queue")
    except Exception as e:
        print(f"----- Failed to add runs to the annotation queue: {e}")
else:
    print("----- No results to add to the annotation queue")

----- New dataset created: name='prompt_enhance_example2' description=None data_type=<DataType.kv: 'kv'> id=UUID('56e3e8cd-6e4d-4904-b950-ba63c45b5533') created_at=datetime.datetime(2025, 8, 17, 8, 45, 17, 756217, tzinfo=datetime.timezone.utc) modified_at=datetime.datetime(2025, 8, 17, 8, 45, 17, 756217, tzinfo=datetime.timezone.utc) example_count=0 session_count=0 last_session_start_time=None inputs_schema=None outputs_schema=None transformations=None
----- Added new examples successfully (bbf419d2-abe7-471b-880b-d5a61b08f687)
----- Run chain on new example
View the evaluation results for project 'flowery-pencil-5' at:
https://smith.langchain.com/o/b0bf207d-6152-41bc-8620-1d01b85129e7/datasets/56e3e8cd-6e4d-4904-b950-ba63c45b5533/compare?selectedSessions=312906d9-81de-4e97-aca7-897ff66401db

View all tests for Dataset prompt_enhance_example2 at:
https://smith.langchain.com/o/b0bf207d-6152-41bc-8620-1d01b85129e7/datasets/56e3e8cd-6e4d-4904-b950-ba63c45b5533
[---------------------------

KeyboardInterrupt: 

### 2. LangSmith에서 피드백 

* https://smith.langchain.com/

### 3. 프롬프트 개선

In [ ]:
def combine_feedback_by_run_id(feedback_list):
    new_feedbacks = {}
    for item in feedback_list:
        run_id = str(item.run_id)
        if run_id in new_feedbacks:
            selected_feedback = new_feedbacks[run_id]
        else:
            selected_feedback = {"correctness": 0, "score": 0, "comment": ""}
            new_feedbacks[run_id] = selected_feedback

        key = item.key
        if key == "correctness":
            selected_feedback["correctness"] = item.score
        elif key == "score":
            selected_feedback["score"] = item.score
        elif key == "note":
            selected_feedback["comment"] = item.comment

    print(f"new_feedbacks = {new_feedbacks}")

    # XML 형식으로 변환
    result = []
    for _key, value in new_feedbacks.items():
        feedback_xml = f"""<feedback>
            <correctness>{value['correctness']}</correctness>
            <score>{value['score']}/5.0</score>
            <comment>{value['comment']}</comment>
            </feedback>"""
        result.append(feedback_xml)
        print(f"feedback_xml = {feedback_xml}\n\n")

    return result
    
#############################################################################
# 3. 프롬프트 최적화(optimization) 단계
# annotation queue 에서 feedback 추출 -> 피드백을 바탕으로 프롬프트 최적화
#############################################################################
# 3-1. 피드백을 가져와서 텍스트로 변환
print("Getting feedbacks -------------------------------------------------")
res = client.list_feedback(run_ids=run_ids)
print("Parsing feedbacks -------------------------------------------------")
feedback_list = combine_feedback_by_run_id(res)

In [ ]:
# 3-2. 현재 프롬프트, 기사 원문, 요약 결과, 피드백 데이터를 사용해서
#   프롬프트 최적화 chain 실행
optimizer = optimizer_prompt | optimizer_llm | StrOutputParser()
print("Optimize prompt -------------------------------------------------")
optimized = optimizer.invoke(
    {
        "instruction": instruction,
        "human_feedback": "\n\n".join(feedback_list),
    }
)
print(f"Optimized = \n{optimized}\n")

In [ ]:
new_summary = summary_chain.invoke({"news":news.page_content[:1000], "instruction":optimized})
print(new_summary)